In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Week5Project import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "sawyer"
password = "12345"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px', 'height': '100px'}),
    html.H1('Sawyer Hannel'),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Wild'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        multi=False,
        value='Reset',
        style={'width':'40%'}
    ),
    ),

       
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        
    html.Hr(),
    dt.DataTable(
            id='datatable-id',
            editable=False,
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            column_selectable=False,
            row_selectable=False,
            row_deletable=False,
            selected_columns=[],
            selected_rows=[],
            page_action="native",
            page_current= 0,
            page_size= 10,
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
    html.Hr(),
        
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6', 
            children = [
                dcc.Graph(id="graph")
                ]
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    df2 = pd.DataFrame.from_records(shelter.read({}))

    
    if filter_type == 'Water':
        dff = df2.breed.isin(["Newfoundland", "Chesapeake Bay Retriever", "Labrador Retriever Mix"])\
        &df2.sex_upon_outcome.isin(["Intact Female"])& df2.age_upon_outcome_in_weeks.ge(26)\
        &df2.age_upon_outcome_in_weeks.le(156)
        
        filterd = df2[dff]
        data=filterd.to_dict('records')
        
    elif filter_type == 'Wild':  
        
        dff = df2.breed.isin(["German Shepherd", "Alaskan Malamute", "Old English Sheepdog","Siberian Husky","Rotteiler"])\
        &df2.sex_upon_outcome.isin(["Intact Mmale"]) & df2.age_upon_outcome_in_weeks.ge(26)\
        &df2.age_upon_outcome_in_weeks.le(156)
        
        filterd = df2[dff]
        data=filterd.to_dict('records')
        
    elif filter_type == 'Disaster':  
        dff = df2.breed.isin(["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"])\
        &df2.sex_upon_outcome.isin(["Intact Male"]) & df2.age_upon_outcome_in_weeks.ge(20)\
        &df2.age_upon_outcome_in_weeks.le(301)
        
        filterd = df2[dff]
        data=filterd.to_dict('records')  
        
    elif filter_type == 'Reset':  
        data=df2.to_dict('records')  
        
        
    columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df2.columns
        ]    
### FIX ME Add code to filter interactive data table with MongoDB queries 
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    dff = pd.DataFrame.from_dict(viewData)
    df2 = dff.iloc[dff['breed']]
    # add code for chart of your choice (e.g. pie chart) #
    return dcc.Graph(figure = px.pie(df2, values="breed", names="breed", title="Breeds")) 
    

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '800px', 'height': '400px'}, center=[dff.iloc[1,13],dff.iloc[1,14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[1,13],dff.iloc[1,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app

Connected
